# PREP ACTIVITY
## Part Two: Computational Method

The purpose of this portion of the Prep Activity is to get you exposed to the computational equivalent of your analytical solution, as well as common plotting and masking techniques.

Lets start by putting your name down, and importing some libraries. Note that if you are using Jupyter, you want to use the `%matplotlib notebook` and if you are using VS Code, you want to use `%matplotlib`.

In [1]:
#Enter Name Here: 

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib
#%matplotlib notebook

Using matplotlib backend: QtAgg


**Now lets define some quantities and constants:**

These quantities are tied to the dielectric material:

- `R`:   Radius of the Dielectric Water Sphere
- `ds`:  Discretized Step Size Over all Space
- `X_e`: Dielectric Constant of Water
- `e_0`: Permittivity of Free Space

These Quantities are tied to the Conductor Shape Parameters:

- `r`:           Radius of the Conducting Sphere
- `V`: Potential of the Surface of the Conductor

All of these quantities are free to be changed.  

In [31]:
R = 1
ds = 0.1
x_e = 80.1
e_0 = 8.85e-12
r = 0.5
V = 5

**Our next step is going to be creating the environment in which all the compuation is occuring within. To due this, we will utilize `np.meshgrid` to generate (x,y,z) spatial coordinates**

In [32]:
x = np.arange(-2*R,2*R+ds,ds)
y = np.arange(-2*R,2*R+ds,ds)
z = np.arange(-2*R,2*R+ds,ds)
X,Y,Z = np.meshgrid(x,y,z,indexing='ij')   

**In order to be able to calculate the potential everywhere within the environment, a separate potential array will be needed to store the potential values at each location in space**

This will now link the potential array to the `np.meshgrid`

In [33]:
volt = np.zeros((x.size,y.size,z.size),dtype = np.float64)

Following the creation of the potential array and the environment, some masks need to be created in order to index into two different regions of space: one where you are within the dielectric sphere and another when you are outside the dielectric sphere

In [34]:
di_out_mask = np.where(X**2 + Y**2 + Z**2 >= R**2) 
di_in_mask = np.where(X**2 + Y**2 + Z**2 <= R**2)  

**Now lets define a function that can initialize potential values along the conductor's interior and surface.**

The function require three input variables: The radius of the dielectric, the radius of the conducting sphere, and the potential array. Similar to `di_out_mask`, we can utilize a mask to assign the potential value `V` to the conducting sphere.

In [35]:
def cond_sphere(R,r,volt):
    if r > R:
        print('Your chosen sphere is too large')
        return None
    sphere_index = np.where(((X**2 + Y**2 + Z**2) <= (r**2)))
    volt[sphere_index] = V
    return sphere_index, volt

### The Method of Relaxation

The method of relaxation is how we can computationally calculate the potential everywhere within the environment. It is essentially an averaging scheme that computes the potential at a given location in space by averaging the potential values all around it. This averaging scheme needs to be repeated in order to stabilize the potential values. Due to this repetition due to the `while` loop, the intial potential within the conducting sphere must be re-initialized per iteration.

Special thanks to Vincent Vaughn-Uding for assistance with the creation of `relax`

In [36]:
def relax(volt,index, num):
    count = 0
    while(num != count):
        old_volt = volt                                                                                                   
        volt[index] = V                                                                                                        
        for i in range(1,volt.shape[0]-1):
            for j in range(1,volt.shape[1]-1):
                for k in range(1,volt.shape[2]-1):
                    volt[i,j,k] =  (old_volt[i+1,j,k] +  old_volt[i-1,j,k] +  old_volt[i,j+1,k] +  
                                    old_volt[i,j-1,k] + old_volt[i,j,k+1] +  old_volt[i,j,k-1])/6
        count = count + 1
    return volt

**For plotting practice, use a 3D Matplotlib Scatter Plot to visualize the space coordinates of the universe and of the dielectric sphere.**

*Hint: Use a mask to select the spatial coordinates correlated to only the dielectric sphere for plotting the dielectric sphere*

In [37]:
#Type your answer here

In [38]:
#Solution for Environment Plot
fig = plt.figure()
plt.title(f'Universe in Cartesian Coordinates with sidelength {4*R} [m]')
ax = fig.add_subplot(projection='3d')
ax.scatter(X, Y, Z,color = 'black')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_aspect('equal')
ax.set_xlim(-2*R,2*R)
ax.set_ylim(-2*R,2*R)
ax.set_zlim(-2*R,2*R)
plt.show()

In [39]:
#Solution for Dielectric Sphere Plot
fig = plt.figure()
plt.title(f'Water Dielectric in Cartesian Coordinates with radius {R} [m]')
ax = fig.add_subplot(projection='3d')
ax.scatter(X[di_in_mask], Y[di_in_mask], Z[di_in_mask],color = 'blue')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_aspect('equal')
ax.set_xlim(-2*R,2*R)
ax.set_ylim(-2*R,2*R)
ax.set_zlim(-2*R,2*R)
plt.show()

### Sphere Potential Calculations

Now that you have successfully plotting spatial coordinates using masks, lets get to the computation of the potential. The first step will be to generate a sphere from the `cond_sphere` function and calculate the potential everywhere in space using `relax`.

In [40]:
#Type your answer here

In [41]:
#Solution
sphere_mask, sphere_volt = cond_sphere(R,r,volt)
new_sphere_volt = relax(sphere_volt,sphere_mask,100)

Just to double check that our sphere mask worked, lets plot the sphere

In [42]:
fig = plt.figure()
plt.title(f'Sphere in Cartesian Coordinates with radius {r} [m]')
ax = fig.add_subplot(projection='3d')
ax.scatter(X[sphere_mask], Y[sphere_mask], Z[sphere_mask],color = 'black')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_aspect('equal')
ax.set_xlim(-R,R)
ax.set_ylim(-R,R)
ax.set_zlim(-R,R)
plt.show()

**Since the potential is now calculated everywhere in space, lets use a contour plot to check it out**

In [43]:
sph_len = sphere_volt.shape[2]
sph_mid_len = int(round(sph_len,0)/2)
plt.title(f'Equipotential Contour Plot of Sphere with radius {r} [m]')
plt.contourf(X[:,:,sph_mid_len],Y[:,:,sph_mid_len],new_sphere_volt[sph_mid_len,:,:],levels = 5000,cmap = 'hot')
plt.tight_layout()
plt.colorbar(label = 'potential [V]',ticks = np.linspace(0,V,V))
plt.show()

Locator attempting to generate 5000 ticks ([0.0, ..., 4.999]), which exceeds Locator.MAXTICKS (1000).


In order to visualize the potential using a contour plot in 2D, one of the position variables needs to be fixed. Which one is being fixed in this situation? What is `sph_mid_len` doing? Would fixing other positions instead have an impact on the contour plot?

In [44]:
#Your answer here

### Electric Field Sphere Calculations

In order to calculate the electric field, we need to create two separate potential arrays. One that is for inside the dielectric and the other for the outside of the dielectric. This is because the electric field is different for these two regions and since we are utilizing `np.gradient` to calculate the electric field, we need two separate potentials.

In [45]:
V_in_sph = np.zeros((new_sphere_volt.shape),dtype = np.float64) 
V_in_sph[di_in_mask] = new_sphere_volt[di_in_mask]             

**The Electric Field Inside the Dielectric:**

In [46]:
E_sph_in = np.gradient(V_in_sph) * -np.ones((V_in_sph.shape),dtype = np.float64) / (np.ones((V_in_sph.shape),dtype = np.float64)*x_e)   #computes electric field just within the dielectric. It is zero outside
x_E_sph_in = E_sph_in[0]        #E_x vector inside of dielectric
y_E_sph_in = E_sph_in[1]        #E_y vector inside of dielectric
z_E_sph_in = E_sph_in[2]        #E_z vector inside of dielectric

**The Electric Field Outside the Dielectric:**

In [47]:
E_sph_out =  np.gradient(new_sphere_volt) * -np.ones((new_sphere_volt.shape),dtype = np.float64)        #computes the negative gradient of the potential everywhere, including the inside, but the inside computation is incorrect
x_E_sph_out = E_sph_out[0]      #E_x vector outside of dielectric
y_E_sph_out = E_sph_out[1]      #E_y vector outside of dielectric
z_E_sph_out = E_sph_out[2]      #E_z vector outside of dielectric

**3D Electric Field Plot**

In [48]:
ax = plt.figure().add_subplot(projection='3d')
plt.title(f"Electric Field Sphere with potential {V} [V] and radius {r} [m]")
ax.quiver(X[di_in_mask], Y[di_in_mask], Z[di_in_mask], x_E_sph_in[di_in_mask], y_E_sph_in[di_in_mask], z_E_sph_in[di_in_mask], length=.1,arrow_length_ratio=0.5,color = 'black')
ax.quiver(X[di_out_mask], Y[di_out_mask], Z[di_out_mask], x_E_sph_out[di_out_mask], y_E_sph_out[di_out_mask], z_E_sph_out[di_out_mask], length=.1,arrow_length_ratio=0.5,color = 'black')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_aspect('equal')
ax.set_xlim(-2*R,2*R)
ax.set_ylim(-2*R,2*R)
plt.show()

Is this graphical representation of the electric field makes sense? Why or why not? Is there anything confusing about this representation?

In [49]:
#Your answer here

You probabily noticed that this representation really isn't useful. Lets do a 2D representation instead 

**2D Electric Field Plot**

In [50]:
#slices the electric field within dielectric at midpoint
x_in_pl_sph = x_E_sph_in[:, :, sph_mid_len]        
y_in_pl_sph = y_E_sph_in[:, :, sph_mid_len]  

#slices the electric field outside dielectric at midpoint 
x_out_pl_sph = x_E_sph_out[:, :, sph_mid_len]     
y_out_pl_sph = y_E_sph_out[:, :, sph_mid_len]

#slices the positions
X_sph = X[:, :, sph_mid_len]                         
Y_sph = Y[:, :, sph_mid_len]
Z_sph = Z[:, :, sph_mid_len]

#grabs 2D indices of the slices that is within the dielectric
new_index_sph = np.where(X_sph**2 + Y_sph**2 + Z_sph**2 < R)   

#grabs 2D indices of the slices that is outside the dielectric
out_index_sph = np.where(X_sph**2 + Y_sph**2 + Z_sph**2 > R)    

#relating color of plot to magnitude of vectors
color_in_sph = np.sqrt(x_in_pl_sph**2 + y_in_pl_sph**2)     
color_out_sph = np.sqrt(x_out_pl_sph**2 + y_out_pl_sph**2)   

fig, ax = plt.subplots()
ax.set_title(f"Cross-sectional Cut of Electric Field Sphere with potential {V} [V] and radius {r} [m]")
ax.set_aspect('equal')
ax.set_facecolor('black')
quiver_1 = ax.quiver(X_sph[new_index_sph],Y_sph[new_index_sph],x_in_pl_sph[new_index_sph],y_in_pl_sph[new_index_sph],color_in_sph[new_index_sph],units='xy',angles='xy',scale_units='xy',headlength=5,cmap='hot')    
quiver_2 = ax.quiver(X_sph[out_index_sph],Y_sph[out_index_sph],x_out_pl_sph[out_index_sph],y_out_pl_sph[out_index_sph],color_out_sph[out_index_sph],units='xy',angles='xy',scale_units='xy',headlength=5,cmap='hot') 
cbar = plt.colorbar(quiver_1,ticks = np.linspace(0,np.max(color_in_sph),5))      
cbar.set_label(r'$|\mathrm{\vec{E}}| \quad [\mathrm{N C^{-1}}]$')
ax.add_patch(plt.Circle((0, 0), R, color='red', fill=False))
ax.add_patch(plt.Circle((0, 0), r, color='red', fill=False))
ax.set_xlim(-2*R,2*R)
ax.set_ylim(-2*R,2*R)

(-2.0, 2.0)

Which position is being held constant when making this 2D Electric Field plot? Does this plot make physical sense? Please elaborate

In [51]:
#Your answer here

### Inner Bound Charge Calculation

In order to calculate the inner bound charge on the interior of the dielectric, we need to index into just outside the conducting sphere's surface. To due this, will will need to make a new mask that can go to the surface of the sphere, index 1 above the sphere, and then remove the contents of the interior of the sphere, only leaving the surface.

Thank you to Vincent Vaughn-Uding for assistance with the creation of the mask.


In [52]:
pos_sphere_mask = np.zeros((x.size,y.size,z.size),dtype = bool)     
pos_sphere_mask[sphere_mask] = True                                   
pos_sphere_mask[1:-1,1:-1,1:-1] = (pos_sphere_mask[2:,1:-1,1:-1] | pos_sphere_mask[0:-2,1:-1,1:-1] 
| pos_sphere_mask[1:-1,2:,1:-1] | pos_sphere_mask[1:-1,0:-2,1:-1] | pos_sphere_mask[1:-1,1:-1,2:] | pos_sphere_mask[1:-1,1:-1,0:-2])
pos_sphere_mask[sphere_mask] = False    

Now, we can use the mask to grab the electric fields just at the surface of the sphere

In [53]:
x_E_sphere_bound_in = x_E_sph_in[pos_sphere_mask]
y_E_sphere_bound_in = y_E_sph_in[pos_sphere_mask]
z_E_sphere_bound_in = z_E_sph_in[pos_sphere_mask]

Lastly, since we are at the surface of the sphere, we can assume that the electric field is anti-parallel to the normal vector at the inner surface of the dielectric. That allows us to not need to take the dot product of the normal vector with the electric field vector. We can just take the negative magnitude of the electric field at the surface.

In [54]:
mag_E_sphere_bound_in_one = np.sqrt(x_E_sphere_bound_in**2 + y_E_sphere_bound_in**2 + z_E_sphere_bound_in**2)
sig_sphere_b = -mag_E_sphere_bound_in_one*e_0*x_e

Now lets plot the surface bound charge density at the surface of the sphere

In [55]:
ax = plt.figure().add_subplot(projection='3d')
plt.title(f'Surface Charge at Inner Boundary with Conducting Sphere of Radius {round(r,4)} [m] and potential {round(V,4)} [V]')
plot = ax.scatter(X[pos_sphere_mask],Y[pos_sphere_mask],Z[pos_sphere_mask],marker = 'o',cmap = 'hot' ,c= sig_sphere_b, s = 1000,alpha = 1)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_aspect('equal')
cbar = plt.colorbar(plot,ticks = np.linspace(0,np.max(sig_sphere_b),10))  
cbar.set_label(r'$\mathrm{\sigma \quad [\frac{C}{m^2}]}$')    
plt.show()

Does this plot make physical sense? Why or why not?

In [56]:
#Your answer here

### Outer Bound Charge Calculation

Similar to how we found the Inner Bound Charge of the Conducting Sphere, we also want to make a mask for just on the inside of the dielectric. To do this, we will use a similar method as used when we increased our index size in order to get to the surface, but we will use the stepsize instead. This method is more preferrable here because it allows us to choose how thick we want our dielectric surface to be. 

In [57]:
scale = 2  
di_surf_mask = np.where(((np.abs(X))**2 + (np.abs(Y))**2 + (np.abs(Z))**2 < (R)**2) & ((np.abs(X)+scale*ds)**2 + (np.abs(Y)+scale*ds)**2 + (np.abs(Z)+scale*ds)**2 > (R+scale*ds)**2))

Now we can use the `di_surf_mask` to grab the electric field just on the inside of the dielectric

In [58]:
x_E_sph_bound_out = x_E_sph_out[di_surf_mask]
y_E_sph_bound_out = y_E_sph_out[di_surf_mask]
z_E_sph_bound_out = z_E_sph_out[di_surf_mask]

Since the conducting sphere is symmetrical with the dielectric sphere, we can assume again that the electric field is parallel to the normal surface vector of the dielectric sphere

In [59]:
mag_E_sphere_bound_out_one = np.sqrt((x_E_sph_bound_out)**2 + (y_E_sph_bound_out)**2 + (z_E_sph_bound_out)**2)
sig_sphere_out = mag_E_sphere_bound_out_one*e_0*x_e

Now lets plot the surface bound charge density at the edge of the dielectric

In [60]:
ax = plt.figure().add_subplot(projection='3d')
plt.title(f'Surface Charge at Outer Boundary with Conducting Sphere of Radius {round(r,4)} [m] and potential {round(V,4)} [V]')
plot = ax.scatter(X[di_surf_mask],Y[di_surf_mask],Z[di_surf_mask],marker = 'o',cmap = 'hot' ,c= sig_sphere_out, s = 1000,alpha = 1)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_aspect('equal')
cbar = plt.colorbar(plot,ticks = np.linspace(0,np.max(sig_sphere_out),10))  
cbar.set_label(r'$\mathrm{\sigma \quad [\frac{C}{m^2}]}$')    
plt.show()

Does this plot make physical sense? Why or why not?

In [61]:
#Your answer here

### Free Bound Charge Calculation

Using your analytical solution of the free charge for the conducting sphere, code in your solution and make a graphical representation of it, similar to the example of the Inner Bound Charge provided above

In [62]:
#Your answer here

In [63]:
#Solution
sig_sphere_f = mag_E_sphere_bound_in_one*e_0
ax = plt.figure().add_subplot(projection='3d')
plt.title(f'Free Surface Charge for Conducting Sphere of Radius {round(r,4)} [m] and potential {round(V,4)} [V]')
plot = ax.scatter(X[pos_sphere_mask],Y[pos_sphere_mask],Z[pos_sphere_mask],marker = 'o',cmap = 'hot' ,c= sig_sphere_f, s = 1000,alpha = 1)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_aspect('equal')
cbar = plt.colorbar(plot,ticks = np.linspace(0,np.max(sig_sphere_f),10))  
cbar.set_label(r'$\mathrm{\sigma \quad [\frac{C}{m^2}]}$')    
plt.show()